<a href="https://colab.research.google.com/github/Shakespeare1998/graymatics/blob/main/Graymatics_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download the Model File

In [1]:
!wget "https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz"
!tar zxvf inception_v3_2016_08_28_frozen.pb.tar.gz

--2023-03-21 21:37:27--  https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 173.194.192.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88668554 (85M) [application/gzip]
Saving to: ‘inception_v3_2016_08_28_frozen.pb.tar.gz’

inception_v3_2016_0 100%[===================>]  84.56M  41.8MB/s    in 2.0s    

2023-03-21 21:37:29 (41.8 MB/s) - ‘inception_v3_2016_08_28_frozen.pb.tar.gz’ saved [88668554/88668554]

inception_v3_2016_08_28_frozen.pb
imagenet_slim_labels.txt


#Model Graph and Image Tensor

In [2]:
import argparse
import os
from os import listdir
import numpy as np
import tensorflow as tf
import cv2


def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.compat.v1.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph


def read_tensor_from_image_file(file_name,
                                input_height=360,
                                input_width=640,
                                input_mean=0,
                                input_std=255):
  tf.compat.v1.disable_v2_behavior()
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.io.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.io.decode_png(file_reader, channels=3, name="png_reader")
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.io.decode_gif(file_reader, name="gif_reader"))
  elif file_name.endswith(".bmp"):
    image_reader = tf.io.decode_bmp(file_reader, name="bmp_reader")
  else:
    image_reader = tf.io.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0)
  resized = tf.compat.v1.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.compat.v1.Session()
  return sess.run(normalized)


def load_labels(label_file):
  proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
  return [l.rstrip() for l in proto_as_ascii_lines]


#Frame Extractor

In [3]:
!mkdir Frames

In [4]:
model_file = \
    "/content/inception_v3_2016_08_28_frozen.pb"
label_file = "/content/imagenet_slim_labels.txt"
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
input_layer = "input"
output_layer = "InceptionV3/Predictions/Reshape_1"

video = cv2.VideoCapture('/content/modified_video.mp4')

x = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
y = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

#format = cv2.VideoWriter_fourcc(*'mp4v')
#writer = cv2.VideoWriter("final_output.mp4", format, 25.0, (x, y))
j=0

while True:
    # Read a frame from the video
  going, frame = video.read()
  if not going:
        break

  frame = frame[30:329, 170:469]
  cv2.imwrite("/content/Frames/Frame"+ str(j).zfill(7)+".jpg", frame) 
  j+=1 
print(j)

1500


#Alternate Frames

In [ ]:
#try:
format = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter("final_output_2nd.mp4", format, 25.0, (299, 299))
for i in range(0,j-1,2):

    #print(ims)
    #if ims.endswith(".jpg"):
    ims = "Frame"+str(i).zfill(7) +".jpg"
    ims2 = "Frame"+str(i+1).zfill(7) +".jpg"
    
    #print(ims)
    image = cv2.imread("/content/Frames/"+ims)
    image_sec = cv2.imread("/content/Frames/"+ims2)

    file_name = "/content/Frames/"+ims
    model_file = \
      "/content/inception_v3_2016_08_28_frozen.pb"
    label_file = "/content/imagenet_slim_labels.txt"
    input_height = 299
    input_width = 299
    input_mean = 0
    input_std = 255
    input_layer = "input"
    output_layer = "InceptionV3/Predictions/Reshape_1"
    
    graph = load_graph(model_file)
    t = read_tensor_from_image_file(
        file_name,
        input_height=input_height,
        input_width=input_width,
        input_mean=input_mean,
        input_std=input_std)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)

    with tf.compat.v1.Session(graph=graph) as sess:
      results = sess.run(output_operation.outputs[0], {
          input_operation.outputs[0]: t
      })
    results = np.squeeze(results)

    top_k = results.argsort()[-1:][::-1]
    labels = load_labels(label_file)
    for i in top_k:
      text = labels[i] + str(results[i])
      print(text)
      if results[i]>0.8:
        color = (0,255,0)
      else:
        color = (0,0,255)
    #print(labels[1]+str(results[1]))
    
    #text = labels[1]+ str(results[1])
    
    image_updated = cv2.putText(image, text, (10,290), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    image_updated2 = cv2.putText(image_sec, text, (10,290), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.imwrite("/content/Frames/"+ims, image_updated)
    cv2.imwrite("/content/Frames/"+ims2, image_updated2)
    writer.write(image_updated)
    writer.write(image_updated2)

video.release()
writer.release()
cv2.destroyAllWindows()
#except:
#    print("")

#Every 4th Frame

In [6]:
model_file = \
    "/content/inception_v3_2016_08_28_frozen.pb"
label_file = "/content/imagenet_slim_labels.txt"
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
input_layer = "input"
output_layer = "InceptionV3/Predictions/Reshape_1"

video = cv2.VideoCapture('/content/modified_video.mp4')

x = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
y = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

#format = cv2.VideoWriter_fourcc(*'mp4v')
#writer = cv2.VideoWriter("final_output.mp4", format, 25.0, (x, y))
j=0

while True:
    # Read a frame from the video
  going, frame = video.read()
  if not going:
        break

  frame = frame[30:329, 170:469]
  cv2.imwrite("/content/Frames/Frame"+ str(j).zfill(7)+".jpg", frame) 
  j+=1 
print(j)

1500


In [ ]:
#try:
format = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter("final_output_4th.mp4", format, 25.0, (299, 299))
for i in range(0,j,4):

    #print(ims)
    #if ims.endswith(".jpg"):
    ims = "Frame"+str(i).zfill(7) +".jpg"
    ims2 = "Frame"+str(i+1).zfill(7) +".jpg"
    ims3 = "Frame"+str(i+2).zfill(7) +".jpg"
    ims4 = "Frame"+str(i+3).zfill(7) +".jpg"
    
    #print(ims)
    image = cv2.imread("/content/Frames/"+ims)
    image_sec = cv2.imread("/content/Frames/"+ims2)
    image_third = cv2.imread("/content/Frames/"+ims3)
    image_4 = cv2.imread("/content/Frames/"+ims4)

    file_name = "/content/Frames/"+ims
    model_file = \
      "/content/inception_v3_2016_08_28_frozen.pb"
    label_file = "/content/imagenet_slim_labels.txt"
    input_height = 299
    input_width = 299
    input_mean = 0
    input_std = 255
    input_layer = "input"
    output_layer = "InceptionV3/Predictions/Reshape_1"
    
    graph = load_graph(model_file)
    t = read_tensor_from_image_file(
        file_name,
        input_height=input_height,
        input_width=input_width,
        input_mean=input_mean,
        input_std=input_std)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)

    with tf.compat.v1.Session(graph=graph) as sess:
      results = sess.run(output_operation.outputs[0], {
          input_operation.outputs[0]: t
      })
    results = np.squeeze(results)

    top_k = results.argsort()[-1:][::-1]
    labels = load_labels(label_file)
    for i in top_k:
      text = labels[i] + str(results[i])
      print(text)
      if results[i]>0.8:
        color = (0,255,0)
      else:
        color = (0,0,255)
    #print(labels[1]+str(results[1]))
    
    #text = labels[1]+ str(results[1])
    
    image_updated = cv2.putText(image, text, (10,290), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    image_updated2 = cv2.putText(image_sec, text, (10,290), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    image_updated3 = cv2.putText(image_third, text, (10,290), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    image_updated4 = cv2.putText(image_4, text, (10,290), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.imwrite("/content/Frames/"+ims, image_updated)
    cv2.imwrite("/content/Frames/"+ims2, image_updated2)
    cv2.imwrite("/content/Frames/"+ims3, image_updated3)
    cv2.imwrite("/content/Frames/"+ims4, image_updated4)
    writer.write(image_updated)
    writer.write(image_updated2)
    writer.write(image_updated3)
    writer.write(image_updated4)

video.release()
writer.release()
cv2.destroyAllWindows()
#except:
#    print("")